In [1]:
import os
import sys
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Lasso

project_path = os.path.abspath(os.path.join(os.getcwd(), '../'))
if project_path not in sys.path:
    sys.path.append(project_path)
    
# Our own code
from src.data.data_builder import DataBuilder, BettingData, MatchData
from src.data.data_transformer import DataTransformer
from src.data.feature_builder import FeatureBuilder
from src.model.metrics import measure_estimators, regression_accuracy

np.random.seed(42)

In [2]:
# Set up data and create cumulative features

csv_paths = ('data/afl_betting.csv', 'data/ft_match_list.csv')
data_classes = (BettingData, MatchData)

raw_df = DataBuilder(data_classes, csv_paths).concat()
model_df = DataTransformer(raw_df).stack_teams()
fb = FeatureBuilder(model_df)
fb.transform()
team_df = fb.df.dropna()
team_df

team         oppo_team  \
team             year round_number                                       
Adelaide         2010 2                     Adelaide            Sydney   
                      3                     Adelaide         Melbourne   
                      4                     Adelaide           Carlton   
                      5                     Adelaide  Western Bulldogs   
                      6                     Adelaide     Port Adelaide   
                      7                     Adelaide          Richmond   
                      8                     Adelaide   North Melbourne   
                      9                     Adelaide          Brisbane   
                      10                    Adelaide          St Kilda   
                      11                    Adelaide         Fremantle   
                      12                    Adelaide          Hawthorn   
                      13                    Adelaide         Melbourne   
                      14                    Adelaide          Essendon   
                      15                    Adelaide        West Coast   
                      16                    Adelaide           Geelong   
                      17                    Adelaide     Port Adelaide   
                      18                    Adelaide          Richmond   
                      19                    Adelaide  Western Bulldogs   
                      20                    Adelaide          Brisbane   
                      21                    Adelaide       Collingwood   
                      22                    Adelaide          St Kilda   
                 2011 1                     Adelaide          Hawthorn   
                      3                     Adelaide         Fremantle   
                      4                     Adelaide     Port Adelaide   
                      5                     Adelaide           Carlton   
                      6                     Adelaide          St Kilda   
                      7                     Adelaide         Melbourne   
                      8                     Adelaide        Gold Coast   
                      9                     Adelaide       Collingwood   
                      10                    Adelaide          Brisbane   
...                                              ...               ...   
Western Bulldogs 2015 21            Western Bulldogs        West Coast   
                      22            Western Bulldogs   North Melbourne   
                      23            Western Bulldogs          Brisbane   
                      25            Western Bulldogs          Adelaide   
                 2016 1             Western Bulldogs         Fremantle   
                      2             Western Bulldogs          St Kilda   
                      3             Western Bulldogs          Hawthorn   
                      4             Western Bulldogs           Carlton   
                      5             Western Bulldogs          Brisbane   
                      6             Western Bulldogs   North Melbourne   
                      7             Western Bulldogs          Adelaide   
                      8             Western Bulldogs         Melbourne   
                      9             Western Bulldogs               GWS   
                      10            Western Bulldogs       Collingwood   
                      11            Western Bulldogs        West Coast   
                      12            Western Bulldogs     Port Adelaide   
                      13            Western Bulldogs           Geelong   
                      15            Western Bulldogs            Sydney   
                      16            Western Bulldogs          Richmond   
                      17            Western Bulldogs        Gold Coast   
                      18            Western Bulldogs          St Kilda   
                      19            Western Bulldogs

In [4]:
# Set up & split data for models

team_features = pd.get_dummies(team_df.drop(['score', 'oppo_score'], axis=1))
team_labels = pd.Series(team_df['score'] - team_df['oppo_score'], name='score_diff')

In [5]:
team_features

win_odds  line_odds  oppo_win_odds  \
team             year round_number                                       
Adelaide         2010 2                 1.54      -12.5           2.49   
                      3                 1.52      -13.5           2.55   
                      4                 1.81       -2.5           2.01   
                      5                 7.50       40.5           1.09   
                      6                 2.55       12.5           1.52   
                      7                 1.20      -29.5           4.66   
                      8                 2.38        9.5           1.59   
                      9                 2.14        5.5           1.72   
                      10                4.25       26.5           1.23   
                      11                2.89       17.5           1.42   
                      12                3.07       18.5           1.38   
                      13                1.62       -8.5           2.31   
                      14                1.62       -8.5           2.32   
                      15                1.45      -15.5           2.77   
                      16                4.03       24.5           1.25   
                      17                1.34      -20.5           3.29   
                      18                1.64       -8.5           2.27   
                      19                3.42       21.5           1.32   
                      20                1.78       -3.5           2.05   
                      21               12.20       54.5           1.04   
                      22                2.47       12.5           1.55   
                 2011 1                 1.96        0.0           1.94   
                      3                 1.49      -13.5           2.63   
                      4                 1.42      -15.5           2.95   
                      5                 3.95       25.5           1.27   
                      6                 1.77       -4.5           2.10   
                      7                 1.96        2.5           1.85   
                      8                 1.15      -38.5           5.50   
                      9                10.25       51.5           1.05   
                      10                1.26      -24.5           3.90   
...                                      ...        ...            ...   
Western Bulldogs 2015 21                3.60       23.5           1.31   
                      22                2.30        6.5           1.63   
                      23                1.22      -28.5           4.50   
                      25                1.93       -1.5           1.93   
                 2016 1                 1.65       -7.5           2.25   
                      2                 1.25      -25.5           3.95   
                      3                 1.96        1.5           1.88   
                      4                 1.08      -49.5           8.00   
                      5                 1.13      -42.5           6.25   
                      6                 2.05        2.5           1.81   
                      7                 1.72       -4.5           2.15   
                      8                 1.42      -16.5           2.95   
                      9                 3.10       18.5           1.38   
                      10                1.48      -13.5           2.70   
                      11                1.73       -5.5           2.15   
                      12                2.20        5.5           1.70   
                      13                2.40        9.5           1.60   
                      15                3.15       16.5           1.38   
                      16                1.20      -33.5           4.80   
                      17                1.11      -43.5           7.00   
                      18                1.35      -20.5           3.25   
                      19                

In [11]:
estimator = make_pipeline(StandardScaler(), Lasso())

predictions = []

for year in range(2011, 2017):
    X_train = team_features[team_features['year'] < year]
    X_test = team_features[team_features['year'] == year]
    y_train = team_labels.loc[X_train.index]
    y_test = team_labels.loc[X_test.index]
    
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    
    predictions.extend(y_pred)
        
predictions

[4.120010244589008,
 14.216497226826135,
 12.737937120304206,
 -22.629997479103064,
 7.484273208525358,
 -7.152197645394417,
 29.87799874043915,
 -55.819800296607546,
 31.32279411762868,
 -10.238322622771857,
 9.265881999063904,
 -26.616028563960626,
 -49.43210777113004,
 -6.998811993896782,
 11.377108439019027,
 -32.130650002730626,
 16.088595686437348,
 1.408648725163797,
 -40.977298714619245,
 18.35884320037108,
 24.62064120391849,
 -19.226033025675942,
 -5.657114471933867,
 -43.680686134335176,
 -37.96531131386334,
 -20.586726826961332,
 -20.317514530616688,
 17.46357280693845,
 -13.389650355058698,
 -4.743415017309672,
 -31.595972389964487,
 -3.357834882318836,
 -50.451568598789365,
 -2.9993646010041957,
 -37.616062570896844,
 14.10733174414391,
 -41.82351676314214,
 -45.463823343983016,
 -40.98578795490698,
 -1.048547023916926,
 7.8187060554537,
 -89.41729562627172,
 -9.049609544821381,
 -40.16486324940919,
 24.54188436423791,
 -35.33483251172128,
 14.827213853177888,
 21.7025858

In [16]:
pred_df = team_df[team_df['year'] > 2010].assign(predicted_margin=predictions)
home_df = pred_df[pred_df['at_home'] == 1]

In [29]:
betting_df = (home_df.loc[:, ['year', 'round_number', 'team', 'oppo_team']]
             .rename(columns={'team': 'home_team', 'oppo_team': 'away_team'})
              .assign(model='tipresias_betting',
                      predicted_home_margin=home_df['predicted_margin'].round(),
                      home_margin=home_df['score'] - home_df['oppo_score'],
                      predicted_home_win=(home_df['predicted_margin'] > 0).astype(int),
                      home_win=(home_df['score'] >
                                home_df['score']).astype(int),
                      draw=(home_df['score'] == home_df['score']).astype(int))
              .assign(tip_point=lambda x: ((x['predicted_home_win'] == x['home_win']) |
                                           (x['draw'])).astype(int))
             .reset_index(drop=True))
betting_df

,year,round_number,home_team,away_team,model,predicted_home_margin,home_margin,predicted_home_win,home_win,draw,tip_point
0,2011,1,Adelaide,Hawthorn,tipresias_betting,4.0,20.0,1,0,1,1
1,2011,3,Adelaide,Fremantle,tipresias_betting,14.0,-25.0,1,0,1,1
2,2011,6,Adelaide,St Kilda,tipresias_betting,7.0,19.0,1,0,1,1
3,2011,8,Adelaide,Gold Coast,tipresias_betting,30.0,57.0,1,0,1,1
4,2011,10,Adelaide,Brisbane,tipresias_betting,31.0,-40.0,1,0,1,1
5,2011,12,Adelaide,West Coast,tipresias_betting,9.0,-39.0,1,0,1,1
6,2011,15,Adelaide,Sydney,tipresias_betting,-7.0,7.0,0,0,1,1
7,2011,17,Adelaide,Essendon,tipresias_betting,11.0,-11.0,1,0,1,1
8,2011,19,Adelaide,Port Adelaide,tipresias_betting,16.0,32.0,1,0,1,1
9,2011,21,Adelaide,Geelong,tipresias_betting,-41.0,-11.0,0,0,1,1


In [31]:
df = DataTransformer(raw_df).clean()

odds_df = (df.loc[:, ['year', 'round_number', 'home_team', 'away_team']]
              .assign(model='oddsmakers',
                      predicted_home_margin=df['home_line_odds'].round() * -1,
                      home_margin=df['home_score'] - df['away_score'],
                      predicted_home_win=((df['home_win_odds'] < df['away_win_odds']) |
                                          (df['home_line_odds'] < df['away_line_odds']) |
                                          # If odds are all equal, predict home team
                                          ((df['home_win_odds'] == df['away_win_odds']) &
                                           (df['home_line_odds'] == df['away_line_odds']))).astype(int),
                      home_win=(df['home_score'] >
                                df['away_score']).astype(int),
                      draw=(df['home_score'] == df['away_score']).astype(int))
              .assign(tip_point=lambda x: ((x['predicted_home_win'] == x['home_win']) |
                                           (x['draw'])).astype(int)))
odds_df

,year,round_number,home_team,away_team,model,predicted_home_margin,home_margin,predicted_home_win,home_win,draw,tip_point
0,2010,1,Richmond,Carlton,oddsmakers,-16.0,-56.0,0,0,0,1
1,2010,1,Geelong,Essendon,oddsmakers,28.0,31.0,1,1,0,1
2,2010,1,Sydney,St Kilda,oddsmakers,-14.0,-8.0,0,0,0,1
3,2010,1,Brisbane,West Coast,oddsmakers,22.0,32.0,1,1,0,1
4,2010,1,Melbourne,Hawthorn,oddsmakers,-32.0,-56.0,0,0,0,1
5,2010,1,Port Adelaide,North Melbourne,oddsmakers,16.0,14.0,1,1,0,1
6,2010,1,Fremantle,Adelaide,oddsmakers,-2.0,56.0,0,1,0,0
7,2010,1,Western Bulldogs,Collingwood,oddsmakers,8.0,-36.0,1,0,0,0
8,2010,2,Brisbane,Carlton,oddsmakers,20.0,19.0,1,1,0,1
9,2010,2,West Coast,Port Adelaide,oddsmakers,10.0,-3.0,1,0,0,0


In [30]:
pd.concat([betting_df, odds_df]).reset_index()

,index,year,round_number,home_team,away_team,model,predicted_home_margin,home_margin,predicted_home_win,home_win,draw,tip_point
0,0,2011,1,Adelaide,Hawthorn,tipresias_betting,4.0,20.0,1,0,1,1
1,1,2011,3,Adelaide,Fremantle,tipresias_betting,14.0,-25.0,1,0,1,1
2,2,2011,6,Adelaide,St Kilda,tipresias_betting,7.0,19.0,1,0,1,1
3,3,2011,8,Adelaide,Gold Coast,tipresias_betting,30.0,57.0,1,0,1,1
4,4,2011,10,Adelaide,Brisbane,tipresias_betting,31.0,-40.0,1,0,1,1
5,5,2011,12,Adelaide,West Coast,tipresias_betting,9.0,-39.0,1,0,1,1
6,6,2011,15,Adelaide,Sydney,tipresias_betting,-7.0,7.0,0,0,1,1
7,7,2011,17,Adelaide,Essendon,tipresias_betting,11.0,-11.0,1,0,1,1
8,8,2011,19,Adelaide,Port Adelaide,tipresias_betting,16.0,32.0,1,0,1,1
9,9,2011,21,Adelaide,Geelong,tipresias_betting,-41.0,-11.0,0,0,1,1
